In [1]:
import pandas as pd
import string
import os
import numpy as np
from sqlalchemy import create_engine, text

from utils import DbEngine, Queries
from etl import ETL
from encoder import Encoding

### Connect to DB

In [2]:
db_engine = DbEngine(user = 'team77', 
                     password = 'mintic2020.',
                     ip = '172.17.0.2', 
                     port = '5432', 
                     db = 'minjusticia')
engine = db_engine.connect()
queries = Queries(engine)

### Loading original data

In [3]:
data = pd.read_excel('../data/ReincidenciaPospenadosNal201011Junio2020Rev.xlsx', skiprows = 6)

### Execute ETL

In [4]:
etl = ETL(queries)
etl.inmate_static_info(data)
etl.inmate_variable_info(data)

/home/daniel/github/MinJusticia_team77_DS4A_Project/app/etl.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reg['CONDIC_EXPECIONAL'] = data_reg['CONDIC_EXPECIONAL'].fillna('NINGUNO')
/home/daniel/github/MinJusticia_team77_DS4A_Project/app/etl.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reg['madre_gestante'] = 'NA'
/opt/conda/envs/nlp/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

### Encode

In [5]:
encoding = Encoding(queries)
data_2_encode = encoding.get_data()
data_encoded = encoding.one_hot(data_2_encode)
data_encoded

,internoen,genero,nacionalidad,reincidente,anio_nacimiento,estado_civil,nivel_educativo,reconocimiento_etnico,diversidad_sexual,extranjero,...,delito_id_delito_172,delito_id_delito_232,delito_id_delito_262,delito_id_delito_other,tentativa_1,tentativa_2,agravado_1,agravado_2,calificado_1,calificado_2
0,0016C0F299261311C9AD0D08788FDC0B35AF1D34,2,22,2,1984,7,5,5,1,1,...,0,0,1,0,1,0,1,0,1,0
1,001FEE5A33538FF7A1230A92CE34D5ADDEFCB31F,2,22,2,1983,3,2,5,1,1,...,0,1,0,0,1,0,1,0,1,0
2,0024DDDA432DF9C8245B2069451376FBFEB22933,1,22,2,1962,3,3,5,1,1,...,0,1,0,0,1,0,1,0,1,0
3,002DB9225F28646D11E5D6E659AF4E5BD4AA720A,2,22,2,1982,3,5,5,1,1,...,0,1,0,0,1,0,1,0,1,0
4,00316773479F6F8256FAD6029DF9170D4D7A1FAB,2,22,2,1987,7,3,5,1,1,...,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103285,FFEE3630A16ACE48B2A0722E0AA3499F86809707,2,22,2,1990,3,2,5,1,1,...,0,0,0,1,1,0,1,0,1,0
103286,FFEE3630A16ACE48B2A0722E0AA3499F86809707,2,22,2,1990,3,2,5,1,1,...,0,0,0,0,1,0,0,1,0,1
103287,FFF7ADD27607233E4A4D1BCF0689EEE1812E1727,2,22,2,1996,6,2,5,1,1,...,0,0,0,0,1,0,0,1,0,1
103288,FFFC9F1A5227DDD6F78179576170997CAAB63AD7,2,22,2,1953,3,3,5,1,1,...,0,1,0,0,1,0,1,0,1,0
